In [1]:
import pandas as pd
import seaborn as sns

%load_ext google.cloud.bigquery

# QUALIFY speed test

In [16]:
%%bigquery --project nbcu-ds-sandbox-a-001

SELECT adobe_tracking_id
        , report_date
        , row_number() OVER(partition by adobe_tracking_id order by report_date ) as upgrade_row_number -- rank the number of times a user upgrade
        , current_date() -- disable results caching
FROM       
        (
            SELECT adobe_tracking_id
                    , report_date
                    , paying_account_flag as paying_account_flag_today
                    , LAG(paying_account_flag,1) OVER ( partition by adobe_tracking_id order by report_date  ) as paying_account_flag_yestd -- paying flag yesterday
            FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER` 
            WHERE report_date between '2022-01-01' and '2022-01-07' -- don't add report_date since we need to distinguish upgrades from beginnning of time 
        )
WHERE paying_account_flag_today = 'Paying' AND paying_account_flag_yestd = 'NonPaying'
order by 3,2,1

Query is running:   0%|          |

Downloading:   0%|          |

,adobe_tracking_id,report_date,upgrade_row_number,f0_
0,++9P2+WhbQt4rjQWiakE+OtWVZ5gBDESVZlhsFLO/o8=,2022-01-02,1,2023-03-03
1,++BZuALDJvn4Uxxo6fv37L4JkSqKDdsnfBGUu3rOhGc=,2022-01-02,1,2023-03-03
2,++O4U6CfhQofob0caUsUAiWrmBLg/1V1xn7K859sX6Q=,2022-01-02,1,2023-03-03
3,++aIiiCONAa+hayeE4idJI4qgLMak1RT9M76RdEMu+4=,2022-01-02,1,2023-03-03
4,++bwvsqrgOQEDtjMqCdFtQo0QWbRRZgNzBJRTeDJRYI=,2022-01-02,1,2023-03-03
...,...,...,...,...
232286,GLASehsS05Pc8PMPNCtfU6jgQzaS7AoFFADyzqYHB+M=,2022-01-07,2,2023-03-03
232287,eXcU8h+m46HjlaJVUz/n4u005866yleewMshl8heX4M=,2022-01-07,2,2023-03-03
232288,kT2X7Ykx4m9Pry/Z3QsIz4omu0u06VNUGWRORrs5z9s=,2022-01-07,2,2023-03-03
232289,uRoiS67Kr6o55445yaIQAUlIq8v0iWPG0BtNcR4CvU8=,2022-01-07,2,2023-03-03


In [17]:
%%bigquery --project nbcu-ds-sandbox-a-001

with cte as (
SELECT adobe_tracking_id
        , report_date
        , current_date() -- disable results caching
FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER` 
WHERE report_date between '2022-01-01' and '2022-01-07'
QUALIFY (paying_account_flag = 'Paying') and (LAG(paying_account_flag,1) over(partition by adobe_tracking_id order by report_date) = 'NonPaying')
)
SELECT adobe_tracking_id
        , report_date
        , row_number() OVER(partition by adobe_tracking_id order by report_date ) as upgrade_row_number -- rank the number of times a user upgrade
FROM cte
order by 3,2,1

Query is running:   0%|          |

Downloading:   0%|          |

,adobe_tracking_id,report_date,upgrade_row_number
0,++9P2+WhbQt4rjQWiakE+OtWVZ5gBDESVZlhsFLO/o8=,2022-01-02,1
1,++BZuALDJvn4Uxxo6fv37L4JkSqKDdsnfBGUu3rOhGc=,2022-01-02,1
2,++O4U6CfhQofob0caUsUAiWrmBLg/1V1xn7K859sX6Q=,2022-01-02,1
3,++aIiiCONAa+hayeE4idJI4qgLMak1RT9M76RdEMu+4=,2022-01-02,1
4,++bwvsqrgOQEDtjMqCdFtQo0QWbRRZgNzBJRTeDJRYI=,2022-01-02,1
...,...,...,...
232286,GLASehsS05Pc8PMPNCtfU6jgQzaS7AoFFADyzqYHB+M=,2022-01-07,2
232287,eXcU8h+m46HjlaJVUz/n4u005866yleewMshl8heX4M=,2022-01-07,2
232288,kT2X7Ykx4m9Pry/Z3QsIz4omu0u06VNUGWRORrs5z9s=,2022-01-07,2
232289,uRoiS67Kr6o55445yaIQAUlIq8v0iWPG0BtNcR4CvU8=,2022-01-07,2


In [15]:
%%bigquery --project nbcu-ds-sandbox-a-001

SELECT adobe_tracking_id
        , report_date
        , row_number() OVER(partition by adobe_tracking_id order by report_date ) as upgrade_row_number -- rank the number of times a user upgrade
FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER` 
WHERE report_date between '2022-01-01' and '2022-01-07'
QUALIFY (paying_account_flag = 'Paying') and (LAG(paying_account_flag,1) over(partition by adobe_tracking_id order by report_date) = 'NonPaying')
order by 3,2,1

Query is running:   0%|          |

Downloading:   0%|          |

,adobe_tracking_id,report_date,upgrade_row_number
0,++9P2+WhbQt4rjQWiakE+OtWVZ5gBDESVZlhsFLO/o8=,2022-01-02,2
1,++BZuALDJvn4Uxxo6fv37L4JkSqKDdsnfBGUu3rOhGc=,2022-01-02,2
2,++O4U6CfhQofob0caUsUAiWrmBLg/1V1xn7K859sX6Q=,2022-01-02,2
3,++aIiiCONAa+hayeE4idJI4qgLMak1RT9M76RdEMu+4=,2022-01-02,2
4,++bwvsqrgOQEDtjMqCdFtQo0QWbRRZgNzBJRTeDJRYI=,2022-01-02,2
...,...,...,...
232286,zytjwALhwovNtYv1jrNBivgYnpB0P8x4lXqbE0Jbr24=,2022-01-07,7
232287,zz6VfVUYUU048zOqEiQ6okOuDJ07Rv5MVI1hVgWHjqI=,2022-01-07,7
232288,zz7kz8DR4vbk37Z6qzCdA/3gJ+2ld66sEEPPfX0fkSA=,2022-01-07,7
232289,zzQ7pCcZjrj1HYboqlJIGWHjjOJLfufzzA8oeJPXXqU=,2022-01-07,7


# Transactional filter

In [3]:
%%bigquery --project nbcu-ds-sandbox-a-001
SELECT distinct campaign_name
FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE`
where lower(campaign_name) like 'transactional%'

Query is running:   0%|          |

Downloading:   0%|          |

,campaign_name
0,TransactionalTermsOfUseEngaged20221028
1,Transactional_ProductAnnualRenewal_20220805
2,Transactional_OSDeprecation_20210218
3,TransactionalTermsOfUseNONEngaged20221028
4,TransactionalFamilyControlProfile20221209


In [12]:
%%bigquery --project nbcu-ds-sandbox-a-001
SELECT * 
FROM `nbcu-ds-prod-001.PeacockDataMartSilver.INFORMATION_SCHEMA.COLUMNS`
where table_name = 'SILVER_MPARTICLE_BRAZE'

Query is running:   0%|          |

Downloading:   0%|          |

,table_catalog,table_schema,table_name,column_name,ordinal_position,is_nullable,data_type,is_generated,generation_expression,is_stored,is_hidden,is_updatable,is_system_defined,is_partitioning_column,clustering_ordinal_position,collation_name,column_default,rounding_mode
0,nbcu-ds-prod-001,PeacockDataMartSilver,SILVER_MPARTICLE_BRAZE,household_id,1,YES,STRING,NEVER,None,None,NO,None,NO,NO,1,NULL,NULL,None
1,nbcu-ds-prod-001,PeacockDataMartSilver,SILVER_MPARTICLE_BRAZE,adobe_tracking_id,2,YES,STRING,NEVER,None,None,NO,None,NO,NO,<NA>,NULL,NULL,None
2,nbcu-ds-prod-001,PeacockDataMartSilver,SILVER_MPARTICLE_BRAZE,campaign_name,3,YES,STRING,NEVER,None,None,NO,None,NO,NO,3,NULL,NULL,None
3,nbcu-ds-prod-001,PeacockDataMartSilver,SILVER_MPARTICLE_BRAZE,campaign_variant_name,4,YES,STRING,NEVER,None,None,NO,None,NO,NO,<NA>,NULL,NULL,None
4,nbcu-ds-prod-001,PeacockDataMartSilver,SILVER_MPARTICLE_BRAZE,campaign_tags,5,YES,STRING,NEVER,None,None,NO,None,NO,NO,<NA>,NULL,NULL,None
5,nbcu-ds-prod-001,PeacockDataMartSilver,SILVER_MPARTICLE_BRAZE,message_variant_channel,6,YES,STRING,NEVER,None,None,NO,None,NO,NO,<NA>,NULL,NULL,None
6,nbcu-ds-prod-001,PeacockDataMartSilver,SILVER_MPARTICLE_BRAZE,canvas_name,7,YES,STRING,NEVER,None,None,NO,None,NO,NO,<NA>,NULL,NULL,None
7,nbcu-ds-prod-001,PeacockDataMartSilver,SILVER_MPARTICLE_BRAZE,canvas_variant_name,8,YES,STRING,NEVER,None,None,NO,None,NO,NO,<NA>,NULL,NULL,None
8,nbcu-ds-prod-001,PeacockDataMartSilver,SILVER_MPARTICLE_BRAZE,canvas_step_name,9,YES,STRING,NEVER,None,None,NO,None,NO,NO,<NA>,NULL,NULL,None
9,nbcu-ds-prod-001,PeacockDataMartSilver,SILVER_MPARTICLE_BRAZE,canvas_tags,10,YES,STRING,NEVER,None,None,NO,None,NO,NO,<NA>,NULL,NULL,None


# Email Engager Discrepancy

### Jan

In [38]:
%%bigquery --project nbcu-ds-sandbox-a-001
SELECT  COUNT(DISTINCT identity)
FROM `nbcu-sdp-prod-003.sdp_persistent_views.BrazeMarketingView`
WHERE (eventName = 'Email Opens')
AND (machineOpen IS NULL)
AND date(eventTimestamp) BETWEEN DATE_SUB('2023-01-31', INTERVAL 1 YEAR) AND '2023-01-31'
AND lower(campaignName) NOT LIKE 'transactional%' -- Exclude transactional emails


Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,50126984


### Feb

In [39]:
%%bigquery --project nbcu-ds-sandbox-a-001
SELECT  COUNT(DISTINCT identity)
FROM `nbcu-sdp-prod-003.sdp_persistent_views.BrazeMarketingView`
WHERE (eventName = 'Email Opens')
AND (machineOpen IS NULL)
AND date(eventTimestamp) BETWEEN DATE_SUB('2023-02-28', INTERVAL 1 YEAR) AND '2023-02-28'
AND lower(campaignName) NOT LIKE 'transactional%' -- Exclude transactional emails


Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,38570038


## All Opens

In [36]:
%%bigquery --project nbcu-ds-sandbox-a-001
SELECT COUNT(DISTINCT identity)
FROM  `nbcu-sdp-prod-003.sdp_persistent_views.BrazeMarketingView` 
WHERE eventName = 'Email Opens'
AND date(eventTimestamp) BETWEEN DATE_SUB('2023-01-31', INTERVAL 1 YEAR) AND '2023-01-31'
AND lower(campaignName) NOT LIKE 'transactional%' -- Exclude transactional emails


Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,60164590


In [37]:
%%bigquery --project nbcu-ds-sandbox-a-001
SELECT COUNT(DISTINCT identity) 
FROM  `nbcu-sdp-prod-003.sdp_persistent_views.BrazeMarketingView` 
WHERE (eventName = 'Email Opens')
AND date(eventTimestamp) BETWEEN DATE_SUB('2023-02-28', INTERVAL 1 YEAR) AND '2023-02-28'
AND lower(campaignName) NOT LIKE 'transactional%' -- Exclude transactional emails


Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,59949647


## Hard Dates

In [42]:
%%bigquery --project nbcu-ds-sandbox-a-001
SELECT COUNT(DISTINCT identity)
FROM  `nbcu-sdp-prod-003.sdp_persistent_views.BrazeMarketingView` 
WHERE (eventName = 'Email Opens')
AND (machineOpen IS NULL)
AND date(eventTimestamp) BETWEEN '2022-01-31' AND '2023-01-31'
AND lower(campaignName) NOT LIKE 'transactional%' -- Exclude transactional emails


Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,50126984


In [43]:
%%bigquery --project nbcu-ds-sandbox-a-001
SELECT COUNT(DISTINCT identity) 
FROM  `nbcu-sdp-prod-003.sdp_persistent_views.BrazeMarketingView` 
WHERE (eventName = 'Email Opens')
AND (machineOpen IS NULL)
AND date(eventTimestamp) BETWEEN '2022-02-28' AND '2023-02-28'
AND lower(campaignName) NOT LIKE 'transactional%' -- Exclude transactional emails


Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,38570038


### Email Acitivity History

In [45]:
%%bigquery --project nbcu-ds-sandbox-a-001

CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Activity_Summary` AS
SELECT  DATE_TRUNC(DATE(eventTimestamp),MONTH)                                                                          AS month
       ,COUNT(DISTINCT CASE WHEN (eventName = 'Email Opens') THEN identity else null end)                               AS unique_all_email_opens
       ,COUNT(DISTINCT CASE WHEN (eventName = 'Email Opens') AND (machineOpen is null) THEN identity else null end)     AS unique_organic_email_opens
--       ,COUNT(DISTINCT CASE WHEN (eventName = 'Email Opens') AND (machineOpen is null) THEN bid else null end)          AS unique_organic_email_opens_adobe_id
       ,COUNT(DISTINCT CASE WHEN (eventName = 'Email Opens') AND (machineOpen is not null) THEN identity else null end) AS unique_machine_opens
--       ,COUNT(DISTINCT CASE WHEN (eventName = 'Email Opens') AND (machineOpen is not null) THEN bid else null end)      AS unique_machine_opens_adobe_id
       ,COUNT(DISTINCT CASE WHEN (eventName = 'Email Sends') THEN identity else null end)                               AS unique_sends
--       ,COUNT(DISTINCT CASE WHEN (eventName = 'Email Sends') THEN bid else null end)                                    AS unique_sends_adobe_id
       ,COUNT(DISTINCT CASE WHEN (eventName = 'Email Deliveries') THEN identity else null end)                          AS unique_deliveries
--       ,COUNT(DISTINCT CASE WHEN (eventName = 'Email Deliveries') THEN bid else null end)                               AS unique_deliveries_adobe_id
       ,COUNT(DISTINCT CASE WHEN (eventName = 'Email Bounces') THEN identity else null end)                             AS unique_bounces
--       ,COUNT(DISTINCT CASE WHEN (eventName = 'Email Bounces') THEN bid else null end)                                  AS unique_bounces_adobe_id
FROM `nbcu-sdp-prod-003.sdp_persistent_views.BrazeMarketingView` a
--LEFT JOIN `nbcu-ds-sandbox-a-001.SLi_sandbox.Braze_Id_Adobe_Id_Map` b ON b.bid = a.identity
WHERE lower(campaignName) NOT LIKE 'transactional%' 
AND DATE_TRUNC(DATE(eventTimestamp),MONTH) BETWEEN '2021-08-28' and '2023-02-28'-- Exclude transactional emails
GROUP BY  1

Query is running:   0%|          |

""


In [47]:
%%bigquery df --project nbcu-ds-sandbox-a-001
select * from `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Activity_Summary` order by month

Query is running:   0%|          |

Downloading:   0%|          |

In [81]:
import matplotlib.pyplot as plt
import plotly.express as px

temp = df[['month', 'unique_organic_email_opens', 'unique_machine_opens']]
temp = temp.set_index('month')
fig = px.line(temp, labels={"month": "Month", "value": "Email Opens"})

newnames = {
"unique_organic_email_opens": "Organic",
"unique_machine_opens": "Machine"
}
fig.update_layout(legend_title="Category")
fig.for_each_trace(lambda t: t.update(name = newnames[t.name], legendgroup = newnames[t.name]))

In [87]:
%%bigquery df2 --project nbcu-ds-sandbox-a-001

SELECT  DATE(eventTimestamp)                                                                      AS month
       ,COUNT(DISTINCT CASE WHEN (eventName = 'Email Opens') THEN identity else null end)                               AS unique_all_email_opens
       ,COUNT(DISTINCT CASE WHEN (eventName = 'Email Opens') AND (machineOpen is null) THEN identity else null end)     AS unique_organic_email_opens
       ,COUNT(DISTINCT CASE WHEN (eventName = 'Email Opens') AND (machineOpen is not null) THEN identity else null end) AS unique_machine_opens
FROM `nbcu-sdp-prod-003.sdp_persistent_views.BrazeMarketingView` a
WHERE DATE(eventTimestamp) BETWEEN '2022-02-01' and '2022-03-31'
AND lower(campaignName) NOT LIKE 'transactional%' -- Exclude transactional emails
GROUP BY  1
order by 1

Query is running:   0%|          |

Downloading:   0%|          |

In [88]:
import matplotlib.pyplot as plt
import plotly.express as px

temp = df2[['month', 'unique_organic_email_opens', 'unique_all_email_opens', 'unique_machine_opens']]
temp = temp.set_index('month')
px.line(temp)

Most opens are machine opens

Open category distinguishing feature began at EoM Feb 22

# Pipeline Duplication Detection

## Holdout Seperation

In [82]:
params = {
    'report_start_date': '2023-01-01',
    'report_end_date': '2023-01-31'
}

In [85]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
WITH 
Holdout AS (
    select distinct holdout.adobe_tracking_id
    from (
    select distinct TrackingId as adobe_tracking_id -- distinct Hold_Out_Type_Current --cohort, count(distinct TrackingId)
    from `nbcu-ds-prod-001.PeacockDataMartMarketingGold.HOLDOUT_GROUP` 
    where cohort = 'January2023' 
    and Hold_Out_Type_Current = 'Owned Email Holdout'
    -- Exclude those who are assigned to Email Holdout but actually received emails
    ) holdout
    left join 
    (SELECT DISTINCT adobe_tracking_id
        FROM  `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE` 
        WHERE event_name = 'Email Deliveries' 
        AND event_date >= @report_start_date
        AND event_date <= @report_end_date 
    ) delivered-- Email Delivered ever 
    on holdout.adobe_tracking_id = delivered.adobe_tracking_id
    and delivered.adobe_tracking_id is null 
)
select count(*) from Holdout

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,3381634


In [86]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
WITH
Holdout AS (
    select distinct holdout.adobe_tracking_id
    from (
    select distinct TrackingId as adobe_tracking_id -- distinct Hold_Out_Type_Current --cohort, count(distinct TrackingId)
    from `nbcu-ds-prod-001.PeacockDataMartMarketingGold.HOLDOUT_GROUP` 
    where cohort = 'January2023' 
    and Hold_Out_Type_Current = 'Owned Email Holdout'
    -- Exclude those who are assigned to Email Holdout but actually received emails
    ) holdout
    left join 
    (SELECT DISTINCT adobe_tracking_id
        FROM  `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE` 
        WHERE event_name = 'Email Deliveries' 
        AND event_date >= @report_start_date
        AND event_date <= @report_end_date 
    ) delivered-- Email Delivered ever 
    on holdout.adobe_tracking_id = delivered.adobe_tracking_id
    where delivered.adobe_tracking_id is null 
)
select count(*) from Holdout

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,3339832


## Joining Duplication Detection

In [90]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params

CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Save_Denom` AS 
    SELECT distinct adobe_tracking_id, date_of_last_view -- no need for max (days_since_last_view), can be saved multiple times, dedup later
    FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_PRIMARY_DEVICES` 
    WHERE report_date BETWEEN @report_start_date AND @report_end_date 
    AND days_since_last_view BETWEEN 15 AND 29 -- this guarantees we are only getting people who have at least past the 'lapsing' phase in the time period.

Query is running:   0%|          |

""


In [152]:
%%bigquery --project nbcu-ds-sandbox-a-001
select count(adobe_tracking_id), count(distinct adobe_tracking_id) from `nbcu-ds-sandbox-a-001.SLi_sandbox.Save_Denom`

Query is running:   0%|          |

Downloading:   0%|          |

,f0_,f1_
0,23859900,22192155


In [93]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params

CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Save_Num` AS 
    SELECT distinct a.adobe_tracking_id      
    FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.Save_Denom` a
    INNER JOIN `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_VIDEO` VIDEO 
        ON a.adobe_tracking_id = VIDEO.adobe_tracking_id 
        AND adobe_date BETWEEN @report_start_date AND @report_end_date
        AND VIDEO.adobe_date BETWEEN DATE_ADD(date_of_last_view, INTERVAL 15 day) AND DATE_ADD(date_of_last_view, INTERVAL 29 day)

Query is running:   0%|          |

""


In [153]:
%%bigquery --project nbcu-ds-sandbox-a-001
select count(adobe_tracking_id), count(distinct adobe_tracking_id) from `nbcu-ds-sandbox-a-001.SLi_sandbox.Save_Num`

Query is running:   0%|          |

Downloading:   0%|          |

,f0_,f1_
0,6731988,6731988


In [94]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params

CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Winback_Denom` AS 
    SELECT distinct adobe_tracking_id, date_of_last_view
    FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_PRIMARY_DEVICES` 
    WHERE report_date BETWEEN @report_start_date AND @report_end_date 
    AND days_since_last_view BETWEEN 30 AND 90-- this guarantees we are only getting people who have at least past the 'lapsing' phase in the time period.

Query is running:   0%|          |

""


In [154]:
%%bigquery --project nbcu-ds-sandbox-a-001
select count(adobe_tracking_id), count(distinct adobe_tracking_id) from `nbcu-ds-sandbox-a-001.SLi_sandbox.Winback_Denom`

Query is running:   0%|          |

Downloading:   0%|          |

,f0_,f1_
0,23751639,23751639


In [95]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params

CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Winback_Num` AS 
    SELECT distinct a.adobe_tracking_id
    FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.Winback_Denom` a
    INNER JOIN `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_VIDEO`  VIDEO 
        ON a.adobe_tracking_id = VIDEO.adobe_tracking_id 
        AND VIDEO.adobe_date BETWEEN @report_start_date AND @report_end_date 
        AND VIDEO.adobe_date BETWEEN DATE_ADD(date_of_last_view, INTERVAL 30 day) 
            AND DATE_ADD(date_of_last_view, INTERVAL 90 DAY)

Query is running:   0%|          |

""


In [155]:
%%bigquery --project nbcu-ds-sandbox-a-001
select count(adobe_tracking_id), count(distinct adobe_tracking_id) from `nbcu-ds-sandbox-a-001.SLi_sandbox.Winback_Num`

Query is running:   0%|          |

Downloading:   0%|          |

,f0_,f1_
0,5663721,5663721


In [96]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params

CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Upgrade_Denom` AS 
    SELECT distinct adobe_tracking_id 
    FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`  USER
    WHERE paying_account_flag = 'NonPaying' 
    AND USER.report_date BETWEEN @report_start_date and @report_end_date

Query is running:   0%|          |

""


In [156]:
%%bigquery --project nbcu-ds-sandbox-a-001
select count(adobe_tracking_id), count(distinct adobe_tracking_id) from `nbcu-ds-sandbox-a-001.SLi_sandbox.Upgrade_Denom`

Query is running:   0%|          |

Downloading:   0%|          |

,f0_,f1_
0,91746405,91746405


In [97]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params

CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Upgrade_Num` AS 
    SELECT  distinct adobe_tracking_id
    FROM
        (SELECT  
                report_date
             , adobe_tracking_id     
         FROM  `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`  USER
         WHERE entitlement_change_flag IN ('Upgrade: Free to Premium'
                                         , 'Upgrade: Free to Premium+'
                                         , 'Upgrade: Premium to Premium+') 
         AND paying_account_flag = 'Paying'                                            
            AND USER.report_date BETWEEN @report_start_date and @report_end_date
        ) 

Query is running:   0%|          |

""


In [157]:
%%bigquery --project nbcu-ds-sandbox-a-001
select count(adobe_tracking_id), count(distinct adobe_tracking_id) from `nbcu-ds-sandbox-a-001.SLi_sandbox.Upgrade_Num`

Query is running:   0%|          |

Downloading:   0%|          |

,f0_,f1_
0,1514290,1514290


In [98]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params

CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Cancel_Save_Denom` AS
    SELECT distinct adobe_tracking_id
    FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER` 
    WHERE paying_account_flag = 'Paying'
    AND auto_renew_flag = 'OFF'
    AND report_date BETWEEN @report_start_date and @report_end_date

Query is running:   0%|          |

""


In [99]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params

CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Cancel_Save_Num` AS 
    SELECT  distinct adobe_tracking_id
    FROM
    (
        SELECT  adobe_tracking_id
            ,report_date
            ,auto_renew_flag                                                                      AS auto_renew_flag_today
            ,LEAD(auto_renew_flag,1) OVER ( partition by adobe_tracking_id ORDER BY report_date ) AS auto_renew_flag_next_day
        FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`
        WHERE report_date BETWEEN @report_start_date AND @report_end_date
        ORDER BY 1, 2 
    )
    WHERE auto_renew_flag_today = 'OFF'
    AND auto_renew_flag_next_day = 'ON'

Query is running:   0%|          |

""


In [102]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params

CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.New_Upgrade_Denom` AS 
    SELECT  distinct adobe_tracking_id
    FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.upgrade_never`
    WHERE report_date BETWEEN @report_start_date AND @report_end_date

Query is running:   0%|          |

""


In [103]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params

CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.New_Upgrade_Num` AS 
    SELECT  distinct adobe_tracking_id
    FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.upgrade_date_rank`
    WHERE upgrade_row_number = 1
    AND report_date BETWEEN @report_start_date AND @report_end_date

Query is running:   0%|          |

""


In [104]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params

CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Paid_Winbacks_Denom` AS 
    SELECT  distinct adobe_tracking_id
    FROM
    (
        SELECT  adobe_tracking_id
            ,report_date
        FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`
        WHERE paying_account_flag = 'NonPaying'
        AND report_date BETWEEN @report_start_date AND @report_end_date 
    )
    WHERE adobe_tracking_id NOT IN ( SELECT distinct adobe_tracking_id FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.upgrade_never` WHERE report_date BETWEEN @report_start_date AND @report_end_date)

Query is running:   0%|          |

""


In [105]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params

CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Paid_Winbacks_Num` AS 
    SELECT  distinct adobe_tracking_id
    FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.upgrade_date_rank`
    WHERE upgrade_row_number > 1
    AND report_date BETWEEN @report_start_date AND @report_end_date

Query is running:   0%|          |

""


In [106]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params

CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.EOM_Paid_Churn_Denom` AS 
    SELECT  distinct adobe_tracking_id
    FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_CHURN`
    WHERE date = @report_end_date
    AND entitlement = 'Paid'

Query is running:   0%|          |

""


In [107]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params

CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.EOM_Paid_Churn_Num` AS 
    SELECT  distinct adobe_tracking_id
    FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_CHURN`
    WHERE date = @report_end_date
    AND entitlement = 'Paid'
    AND Churn_flag = 'Churn'

Query is running:   0%|          |

""


# Silver Churn QA

In [19]:
%%bigquery --project nbcu-ds-sandbox-a-001
SELECT  *
FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_CHURN`
WHERE base_date = '2023-01-31'
and entitlement = 'Paid'
limit 10;

Query is running:   0%|          |

Downloading:   0%|          |

,Date,base_date,original_entitlement,original_entitlement_updated,original_account_tenure,original_acquisition_channel,original_trial_flag,original_tenure_paid_lens,primary_device_name,entitlement,billing_platform,bundling_partner,account_tenure_base_date,Churn_type,tenure_paid_lens,billing_cycle,Churn_flag,adobe_tracking_id,entitlement_updated
0,2023-02-28,2023-01-31,Premium,Premium Paid Bundled,0 - 1 Day,NBCU,False,0-90 days,Non-viewer,Paid,NBCU,Voucher - PokerFacePublic,<90 Days,N/A,0-90 days,ANNUAL,Not Churned,py712oIUc87nRHznGaL1+ZrnCLR+YDVbxWhkEuSqPiY=,Premium Paid Bundled
1,2023-02-28,2023-01-31,Premium,Premium Paid Bundled,0 - 1 Day,NBCU,False,0-90 days,Ios Mobile,Paid,NBCU,Voucher - PokerFacePublic,<90 Days,N/A,0-90 days,ANNUAL,Not Churned,pAUKNbskvQ6HTZd1bMt8EdQRoAr/MAUE1ntGhr9Z8vg=,Premium Paid Bundled
2,2023-02-28,2023-01-31,Premium,Premium Paid Bundled,0 - 1 Day,NBCU,False,0-90 days,Www,Paid,NBCU,Voucher - PokerFacePublic,<90 Days,N/A,0-90 days,ANNUAL,Not Churned,NOh5dJ1BC8kGOc773qJTKqgH3Iz3sCZxt1l9llPg4Hw=,Premium Paid Bundled
3,2023-02-28,2023-01-31,Premium,Premium Paid Bundled,0 - 1 Day,NBCU,False,0-90 days,Www,Paid,NBCU,Voucher - PokerFacePublic,<90 Days,N/A,0-90 days,ANNUAL,Not Churned,oGkWxDOJaLTTUHLd6nV9mQqFmqmQQUIheSqF1dbJc9c=,Premium Paid Bundled
4,2023-02-28,2023-01-31,Premium,Premium Paid Bundled,0 - 1 Day,NBCU,False,0-90 days,Samsung Tv,Paid,NBCU,Voucher - PokerFacePublic,<90 Days,N/A,0-90 days,ANNUAL,Not Churned,IL3J5qcHWF/R7+jwUpcMT38/Q4HkLY6TnNs7gE/zN10=,Premium Paid Bundled
5,2023-02-28,2023-01-31,Premium,Premium Paid Bundled,0 - 1 Day,NBCU,False,0-90 days,Roku,Paid,NBCU,Voucher - PokerFacePublic,<90 Days,N/A,0-90 days,ANNUAL,Not Churned,t7mfMe1KBMe4Qakrai9A2oF2m7xJDi8ZyX8JuOYNjDM=,Premium Paid Bundled
6,2023-02-28,2023-01-31,Premium,Premium Paid Bundled,0 - 1 Day,NBCU,False,0-90 days,Ios Mobile,Paid,NBCU,Voucher - PokerFacePublic,<90 Days,N/A,0-90 days,ANNUAL,Not Churned,QkbaIaMK3jPb6ITabK2/F0hMGYc9g13/vD2aD0q3Hm0=,Premium Paid Bundled
7,2023-02-28,2023-01-31,Premium,Premium Paid Bundled,0 - 1 Day,NBCU,False,0-90 days,Www,Paid,NBCU,Voucher - PokerFacePublic,<90 Days,N/A,0-90 days,ANNUAL,Not Churned,PquU+X6S2YLK1qAGOS+qUwXS17AWYHhe+jb2mm17ewY=,Premium Paid Bundled
8,2023-02-28,2023-01-31,Premium,Premium Paid Bundled,0 - 1 Day,NBCU,False,0-90 days,Samsung Tv,Paid,NBCU,Voucher - PokerFacePublic,<90 Days,N/A,0-90 days,ANNUAL,Not Churned,JHif6qSvwShn5NHHoKa+nghFF11Hg61yCe7tRkgVRn0=,Premium Paid Bundled
9,2023-02-28,2023-01-31,Premium,Premium Paid Bundled,0 - 1 Day,NBCU,False,0-90 days,Ios Mobile,Paid,NBCU,Voucher - PokerFacePublic,<90 Days,N/A,0-90 days,ANNUAL,Not Churned,rWddN2Rbm3DLcHKC2To4zm/yfW4sAKVvOwJqv2IO9tY=,Premium Paid Bundled


In [17]:
%%bigquery --project nbcu-ds-sandbox-a-001
SELECT  churn_flag
       ,COUNT(DISTINCT adobe_tracking_id)
FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_CHURN`
WHERE base_date = '2023-01-31'
and entitlement = 'Paid'
GROUP BY  1

Query is running:   0%|          |

Downloading:   0%|          |

,churn_flag,f0_
0,Not Churned,19993668
1,Churn,1312746


In [13]:
%%bigquery --project nbcu-ds-sandbox-a-001

SELECT  CASE WHEN paying_account_flag = 'Paying' THEN 'Not Churned'  ELSE 'Churned' END AS churn_flag
       ,COUNT(DISTINCT paying_base.adobe_tracking_id)
FROM
(
	SELECT  adobe_tracking_id
	FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`
	WHERE report_date = '2023-01-31'
	AND paying_account_flag = 'Paying' 
) paying_base
LEFT JOIN
(
	SELECT  adobe_tracking_id
	       ,paying_account_flag
	FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`
	WHERE report_date = DATE_ADD('2023-01-31', INTERVAL 30 DAY) 
) churn_status
ON paying_base.adobe_tracking_id = churn_status.adobe_tracking_id
GROUP BY 1

Query is running:   0%|          |

Downloading:   0%|          |

,churn_flag,f0_
0,Not Churned,19932447
1,Churned,1373967


# Churn Window

In [26]:
%%bigquery --project nbcu-ds-sandbox-a-001
SELECT  *
FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_CHURN`
WHERE base_date = '2023-02-28'
and entitlement = 'Paid'
limit 10;

Query is running:   0%|          |

Downloading: |          |

,Date,base_date,original_entitlement,original_entitlement_updated,original_account_tenure,original_acquisition_channel,original_trial_flag,original_tenure_paid_lens,primary_device_name,entitlement,billing_platform,bundling_partner,account_tenure_base_date,Churn_type,tenure_paid_lens,billing_cycle,Churn_flag,adobe_tracking_id,entitlement_updated


# Lookback Window

In [23]:
%%bigquery --project nbcu-ds-sandbox-a-001 

select DATE_SUB(DATETIME_TRUNC('2023-02-01', QUARTER), INTERVAL 4 MONTH), 

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,2022-09-01


# CustomerPermissionView join

In [ ]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.All_Cohorts_v4_Jan23` AS (
    -- email targetable and holdout cohort, users signed up after 2020/8/11
     SELECT distinct a.adobe_tracking_id as aid
     , cohort
     , user.account_type

    FROM         -- Include email engagers only: users with at least one email open in the last year that is not a transactional email
     (
        SELECT DISTINCT identity 
        FROM  `nbcu-sdp-prod-003.sdp_persistent_views.BrazeMarketingView` 
        WHERE eventName = 'Email Opens' 
        AND machineOpen IS NULL
        AND date(eventTimestamp) BETWEEN DATE_SUB(@report_end_date, INTERVAL 1 YEAR) AND @report_end_date
        AND lower(campaignName) NOT LIKE 'transactional%' -- Exclude transactional emails
    ) Email_Engagers --opened email at least once, using other open 
    
    INNER JOIN `nbcu-ds-sandbox-a-001.SLi_sandbox.Braze_Id_Adobe_Id_Map` mapping
    ON mapping.bid = Email_Engagers.identity

    INNER JOIN
        (
            SELECT *, 'Email_Targeted' as cohort from `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Targeted_v4_Jan23`
            UNION ALL
            SELECT *, 'Holdout' as cohort from `nbcu-ds-sandbox-a-001.SLi_sandbox.Holdout_v4_Jan23`
        ) a
    ON a.adobe_tracking_id = mapping.aid
        
    --add attribute: account_type 
    INNER JOIN ( SELECT * FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER` WHERE report_date = @report_end_date ) user
    ON a.adobe_tracking_id = user.adobe_tracking_id 

    -- for after 2021/july, email channel only, take out all abandon MAAs
    INNER JOIN 
    (SELECT DISTINCT adobe_tracking_id FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_PRIMARY_DEVICES` 
        WHERE report_date = @report_end_date
        AND date_of_last_view IS NOT NULL 
    ) abandon_maa 
    ON a.adobe_tracking_id = abandon_maa.adobe_tracking_id

    -- exclude unsubscribed
    LEFT JOIN `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Unsubs_v4_Jan23` unsub
    ON a.adobe_tracking_id = unsub.adobe_tracking_id
    WHERE unsub.adobe_tracking_id IS NULL
)

# Test

In [34]:
%%bigquery --project nbcu-ds-sandbox-a-001
SELECT  cohort
       ,COUNT(aid)
FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.All_Cohorts_v4_Jan23` a
LEFT JOIN `nbcu-ds-sandbox-a-001.SLi_sandbox.Video_Viewing_v4_Jan23` video
ON a.aid = video.adobe_tracking_id
GROUP BY  1

Query is running:   0%|          |

Downloading:   0%|          |

,cohort,f0_
0,Holdout,1790028
1,Email_Targeted,34010112


In [36]:
%%bigquery --project nbcu-ds-sandbox-a-001
select distinct cohort
FROM `nbcu-ds-prod-001.PeacockDataMartMarketingGold.HOLDOUT_GROUP`

Query is running:   0%|          |

Downloading:   0%|          |

,cohort
0,October2021
1,July2020
2,July2021
3,April2022
4,October2022
5,April2023
6,January2022
7,January2023
8,April2021
9,July2022


Holdout comparison

In [ ]:
SELECT  TrackingId aid
FROM `nbcu-ds-prod-001.PeacockDataMartMarketingGold.HOLDOUT_GROUP`
WHERE cohort = 'October2022'
AND Hold_Out_Type_Current = 'Owned Email Holdout'
AND (
    TrackingId IN (
        -- Find subs that registered in the last 4 months in the last 4 months 
        SELECT  adobe_tracking_id
        FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`
        WHERE report_date = '2022-10-01'
        AND date(registration_date) BETWEEN date_add('2022-10-01', interval -4 month) AND '2022-10-01' 
    ) 
    
    OR 
    
    TrackingId IN (
        -- Find subs that have received an email IN the last 4 months 
        SELECT  adobe_tracking_id
        FROM
        (
            SELECT  adobe_tracking_id
                ,last_value(event_name) over(partition by adobe_tracking_id ORDER BY eventTimestamp) category -- identify if the last action a sub took was to unsubscribe
            FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE`
            WHERE date(eventTimestamp) BETWEEN date_add('2022-10-01', interval -4 month) AND '2022-10-01'
            AND event_name in('Email Deliveries', 'Email Unsubscribes') --isolate users who have either received an email or unsubscribed
            AND lower(campaign_name) not like '%transactional%' -- remove transactional emails 
        )
        WHERE category = 'Email Deliveries' -- remove unsubscribers 
    )
)

In [ ]:
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Measurement_Audience_v4_Jan23` AS

SELECT  distinct a.adobe_tracking_id AS aid
       ,cohort
       ,user.account_type
       ,user.account_tenure
FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Engagement_Qualifier_v4_Jan23` base

-- Include only those who received email in the current reporting period or are in holdout
INNER JOIN
    (
        SELECT *, 'Email_Targeted' as cohort from `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Targeted_v4_Jan23`
        UNION ALL
        SELECT *, 'Holdout' as cohort from `nbcu-ds-sandbox-a-001.SLi_sandbox.Holdout_v4_Jan23`
    ) a
ON a.adobe_tracking_id = base.aid

-- for after 2021/july, email channel only, take out all abandon MAAs
INNER JOIN
    (
        SELECT  DISTINCT adobe_tracking_id
        FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_PRIMARY_DEVICES`
        WHERE report_date = @report_end_date
        AND date_of_last_view IS NOT NULL
    ) abandon_maa
ON a.adobe_tracking_id = abandon_maa.adobe_tracking_id

--add attribute: account_type at the end of the reporting period
INNER JOIN
    (
        SELECT  adobe_tracking_id
            ,account_type
            ,account_tenure
        FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`
        WHERE report_date = @report_end_date 
    ) user
ON a.adobe_tracking_id = user.adobe_tracking_id

-- exclude unsubscribed
LEFT JOIN (
    SELECT DISTINCT adobe_tracking_id
    FROM  `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE` 
    WHERE event_name = 'Email Unsubscribes' 
    AND event_date <= @report_end_date 
) unsub
ON a.adobe_tracking_id = unsub.adobe_tracking_id
WHERE unsub.adobe_tracking_id IS NULL

In [ ]:

%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Targeted_v4_Jan23` AS ( -- everyone who have received emails in the month
    SELECT DISTINCT adobe_tracking_id 
    FROM  `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE`
    WHERE event_name = 'Email Deliveries' 
    AND event_date BETWEEN @report_start_date and @report_end_date
    AND lower(campaign_name) NOT LIKE 'transactional%' -- Exclude transactional emails
)

%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Holdout_v4_Jan23` AS ( 
    SELECT  distinct holdout.adobe_tracking_id
    FROM
    (
        SELECT  distinct TrackingId AS adobe_tracking_id -- distinct Hold_Out_Type_Current --cohort, count(distinct TrackingId)
        FROM `nbcu-ds-prod-001.PeacockDataMartMarketingGold.HOLDOUT_GROUP`
        WHERE cohort = format_timestamp('%B%Y', DATETIME_TRUNC(@report_start_date, QUARTER)) -- get cohort name as month of quarter start + year
        AND Hold_Out_Type_Current = 'Owned Email Holdout'
    ) holdout
    -- Exclude those who are assigned to Email Holdout but actually received emails 1 month prior to holdout start (to include cooloff period)
    LEFT JOIN (
        SELECT DISTINCT adobe_tracking_id 
        FROM  `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE`
        WHERE event_name = 'Email Deliveries' 
        AND event_date BETWEEN DATE_SUB(DATETIME_TRUNC(@report_start_date, QUARTER), INTERVAL 1 MONTH) and @report_end_date
        AND lower(campaign_name) NOT LIKE 'transactional%' -- Exclude transactional emails
    ) received
    ON holdout.adobe_tracking_id = received.adobe_tracking_id
    WHERE received.adobe_tracking_id IS NULL
)

%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Engagement_Qualifier_v4_Jan23` AS 

WITH 
email_engagers AS (
    -- Engagement: Deliveries 4 months before start of the holdout period, defined as start of quarter
    SELECT aid
    FROM
        (
            SELECT DISTINCT identity 
            FROM  `nbcu-sdp-prod-003.sdp_persistent_views.BrazeMarketingView` 
            WHERE eventName = 'Email Deliveries'
            AND date(eventTimestamp) BETWEEN DATE_SUB(DATETIME_TRUNC(@report_start_date, QUARTER), INTERVAL 4 MONTH) AND @report_end_date
            AND lower(campaignName) NOT LIKE 'transactional%' -- Exclude transactional emails
        ) engager_base
    INNER JOIN `nbcu-ds-sandbox-a-001.SLi_sandbox.Braze_Id_Adobe_Id_Map` mapping
    ON mapping.bid = engager_base.identity
)
SELECT aid
FROM email_engagers

UNION ALL

SELECT distinct adobe_tracking_id as aid
FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`
WHERE registration_date >= DATE_SUB(DATETIME_TRUNC(@report_start_date, QUARTER), INTERVAL 4 MONTH)

